<a href="https://colab.research.google.com/github/31Wilson13/Taiwan_Convenience_Index/blob/main/Convenience_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

In [5]:
# install.packages('shiny')
install.packages('shinythemes')
install.packages('dbplyr')
install.packages('ggplot2')
install.packages('maptools')
install.packages('rgdal')
install.packages('sf')
install.packages('purrr')
install.packages('maptools')
install.packages('rgdal')
install.packages('dplyr')
install.packages('DT')
install.packages('shinycssloaders')
install.packages('units')
install.packages('readxl')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘sp’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘e1071’, ‘wk’, ‘classInt’, ‘s2’, ‘units’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-

In [6]:
library(shiny)
library(shinythemes)
library(dbplyr)
library(ggplot2)
library(maptools)
library(rgdal)
library(sf)
library(purrr)
library(maptools)
library(rgdal)
library(dplyr)
library(DT)
library(shinycssloaders)
library(units)
library(readxl)

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Please note that 'maptools' will be retired during October 2023,
plan transition at your earliest convenience (see
https://r-spatial.org/r/2023/05/15/evolution4.html and earlier blogs
for guidance);some functionality will be moved to 'sp'.
 Checking rgeos availability: FALSE

Please note that rgdal will be retired during October 2023,
plan transition to sf/stars/terra functions using GDAL and PROJ
at your earliest convenience.
See https://r-spatial.org/r/2023/05/15/evolution4.html and https:/

# 讀取台灣地圖、便利設施座標檔案

In [ ]:
taiwan <- read_excel("Taiwan_Loc_archived.xlsx")
taiwan_town_facility <- read_excel('taiwan_town_facility_final.xlsx')
estate <- data.frame(read_xlsx('estate.xlsx'))
town_sample <- data.frame(read_xlsx('town_sample.xlsx'))
estate_sf <- st_as_sf(estate[,c('Longitude', 'Latitude')], coords = c('Longitude', 'Latitude'), crs = "epsg:4326" )
facility<-read_xlsx("facility.xlsx")
facility_df <- data.frame(facility[,1:6])
facility_df_sf <- st_as_sf(facility_df[,c('Longitude', 'Latitude')], coords = c('Longitude', 'Latitude'), crs = "epsg:4326")

# 計算周邊便利指數方程式

In [ ]:
# 便利指數各係數權重
p <- c(0.16272209,0.06059852 ,0.06059852,0.06059852,0.65548235)

calculate_area_taiwan_v1 <- function(a, x){

  coord_sf <- st_as_sf(a, coords = c('input.Longitude_1', 'input.Latitude_1'), crs = "epsg:4326")
  dist_facility <- st_distance(coord_sf, facility_df_sf)

  range_distance <- set_units(x, m)
  output_table <- data.frame(matrix(nrow = 1, ncol = 0))
  output_table$Hospital <- apply(matrix(array(dist_facility[which(facility_df$Facility == 'Hospital')] < range_distance)), 2, sum)
  output_table$Store <- apply(matrix(array(dist_facility[which(facility_df$Facility == 'Store')] < range_distance)), 2, sum)
  output_table$School <- apply(matrix(array(dist_facility[which(facility_df$Facility == 'School')] < range_distance)), 2, sum)
  output_table$Police <- apply(matrix(array(dist_facility[which(facility_df$Facility == 'Police')] < range_distance)), 2, sum)
  output_table$Post <- apply(matrix(array(dist_facility[which(facility_df$Facility == 'Post')] < range_distance)), 2, sum)

  return(output_table)
}

# 計算周邊500公尺房價方程式

In [ ]:
calculate_estate_price <- function(a, x){
  coord_sf <- st_as_sf(a, coords = c('input.Longitude_1', 'input.Latitude_1'), crs = "epsg:4326")
  dist_estate <- st_distance(coord_sf, estate_sf)
  range_distance <- set_units(x, m)
  price <- mean(estate$Price[which(dist_estate < range_distance)])
  return(price)
}

# Shiny互動式網站

In [ ]:
ui <- fluidPage(
  navbarPage('Shine Your Life', theme =  shinytheme('lumen'),
             tabPanel("台灣地圖", fluid = TRUE, icon = icon("globe-americas"),
                      sidebarLayout(
                        sidebarPanel(
                          titlePanel(h3('地圖篩選條件')),
                          fluidRow(column(5, offset = 0,
                                          checkboxGroupInput(inputId = 'FacilityFinder',
                                                             label = '設施：',
                                                             choices = c('便利商店' = 'Store', '醫院藥局' = 'Hospital', '警察局' = 'Police', '郵局' = 'Post', '國中小' = 'School'),
                                                             selected = 'Store')
                          ),
                          column(6, offset = 0,
                                 checkboxGroupInput(inputId = 'RegionFinder',
                                                    label = '地區：',
                                                    choices = c('基隆市' = 10017, '新北市' = 65000, '臺北市' = 63000, '桃園市' = 68000,
                                                                '新竹縣' = 10004, '新竹市' = 10018, '苗栗縣' = 10005, '臺中市' = 66000, '彰化縣' = 10007,
                                                                '南投縣' = 10008, '雲林縣' = 10009, '嘉義縣' = 10010, '嘉義市' = 10020, '臺南市' = 67000,
                                                                '高雄市' = 64000, '屏東縣' = 10013, '宜蘭縣' = 10002, '花蓮縣' = 10015, '臺東縣' = 10014),
                                                    selected = 63000))),
                          hr(),
                          titlePanel(h3('目標位置')),
                          fluidRow(column(5,
                                          textInput(inputId = 'Latitude_1',
                                                    label = '緯度：',
                                                    value = '25.047797954935447',
                                                    width = '100px')),
                                   column(5,
                                          textInput(inputId = 'Longitude_1',
                                                    label = '經度：',
                                                    value = '121.51742765316871',
                                                    width = '100px'))),
                          helpText('請輸入目標位置的經緯度，標記於地圖上，查看周邊500公尺內便利設施狀況')

                        ),
                        mainPanel(
                          withSpinner(plotOutput('distPlot')),
                          hr(),
                          helpText('下表顯示目標周邊半徑500公尺內便利設施數量與平均單位房價'),
                          DT::dataTableOutput('facilitytableFinder'),
                          hr(),
                          helpText('下表顯示台灣部分地標便利狀況，可做為參考依據'),
                          DT::dataTableOutput('town_sample')

                        )
                      )),
             tabPanel("推薦系統", fluid = TRUE, icon = icon("chart-bar"),
                      sidebarLayout(
                        sidebarPanel(width = 2.5,
                                     titlePanel(h3('偏好設定')),
                                     fluidRow(
                                       selectInput(inputId = 'CityPreferenceFinder',
                                                   label = '城市：',
                                                   choices =  c('新北市' = 65000, '臺北市' = 63000, '桃園市' = 68000),
                                                   selected = 63000),
                                       selectInput(inputId = 'TypePreferenceFinder',
                                                   label = '住宅類型：',
                                                   choices =  c('公寓(5樓含以下無電梯)', '華廈(10層含以下有電梯)', '住宅大樓(11層含以上有電梯)'),
                                                   selected = '住宅大樓(11層含以上有電梯)'),
                                       selectInput(inputId = 'YearPreferenceFinder',
                                                   label = '屋齡：',
                                                   choices =  c('10年以下', '11至30年', '30年以上'),
                                                   selected = '10年以下')
                                     )),
                        mainPanel(width = 9.5,
                                  withSpinner(plotOutput('preferencePlot')),
                                  hr(),
                                  helpText('下圖顯示推薦地點'),
                                  dataTableOutput('preferenceFinder')
                        )
                      )),
             tabPanel("鄉鎮市區", fluid = TRUE, icon = icon("chart-bar"),
                      selectInput(inputId = 'TownFinder',
                                  label = '城市：',
                                  choices =  c('基隆市' = 10017, '新北市' = 65000, '臺北市' = 63000, '桃園市' = 68000,
                                               '新竹縣' = 10004, '新竹市' = 10018, '苗栗縣' = 10005, '臺中市' = 66000, '彰化縣' = 10007,
                                               '南投縣' = 10008, '雲林縣' = 10009, '嘉義縣' = 10010, '嘉義市' = 10020, '臺南市' = 67000,
                                               '高雄市' = 64000, '屏東縣' = 10013, '宜蘭縣' = 10002, '花蓮縣' = 10015, '臺東縣' = 10014),
                                  selected = 63000,
                                  width = '220px'),
                      mainPanel(
                        helpText('表格顯示各地區便利設施總數量與便利指數'),
                        dataTableOutput('townstableFinder')
                      )
             ))
)

server <- function(input, output) {

  city_finder <- reactive({
    req(input$RegionFinder)
    filter(taiwan, ID %in% input$RegionFinder)
  })

  facility_finder <- reactive({
    req(input$RegionFinder)
    req(input$FacilityFinder)
    filter(facility_df, City_ID %in% input$RegionFinder) %>%
      filter(Facility %in% input$FacilityFinder)
  })

  coord_finder <- reactive({
    req(input$Latitude_1)
    req(input$Longitude_1)
    coord_df <- data.frame(input$Latitude_1, input$Longitude_1)
  })

  output$distPlot <- renderPlot({
    input$RegionFinder
    my_sf <- st_as_sf(city_finder(), coords = c('Longitude', 'Latitude'), crs = 4326)
    object_sf <- st_as_sf(coord_finder(), coords = c('input.Longitude_1', 'input.Latitude_1'), crs = "epsg:4326")
    ggplot(my_sf) +
      geom_sf(aes()) +
      geom_point(data=facility_finder(), aes(x = Longitude, y = Latitude, color = Facility), size = 0.5, alpha = 0.7)+
      geom_sf(data = object_sf, fill = 'cyan', size = 8, alpha = 0.7, shape = 21, stroke = 0)+
      scale_color_manual(values = c(c(Store = 'darkkhaki', Hospital = 'pink', Post = 'blue', Police = 'purple', School = 'red')))+
      theme_void()
  })



  output$facilitytableFinder <- DT::renderDataTable({
    input$coord_finder
    facility_table_1 <- calculate_area_taiwan_v1(coord_finder(), 500)
    facility_table_1$Index <- round(sum(facility_table_1[,1:5] * p),2)

    facility_table_1$Price <- round(calculate_estate_price(coord_finder(), 500),2)
    colnames(facility_table_1) <- c('醫院藥局','便利商店','國中小','警察局','郵局', '便利指數', '平均房價/坪')
    DT::datatable(facility_table_1)
  })

  output$town_sample <- DT::renderDataTable({
    DT::datatable(town_sample)
  })

  estate_finder <- reactive({
    req(input$CityPreferenceFinder)
    req(input$TypePreferenceFinder)
    req(input$YearPreferenceFinder)
    filter(estate, City_ID %in% input$CityPreferenceFinder) %>%
      filter(Type %in% input$TypePreferenceFinder) %>%
      filter(Years_Type %in% input$YearPreferenceFinder)
  })

  citypreference_finder <- reactive({
    req(input$CityPreferenceFinder)
    filter(taiwan, ID %in% input$CityPreferenceFinder)
  })

  facilitypreference_finder <- reactive({
    req(input$CityPreferenceFinder)
    filter(facility_df, City_ID %in% input$CityPreferenceFinder)
  })

  output$preferencePlot <- renderPlot({
    input$coord_finder
    my_sf <- st_as_sf(citypreference_finder(), coords = c('Longitude', 'Latitude'), crs = 4326)
    object_sf <- st_as_sf(coord_finder(), coords = c('input.Longitude_1', 'input.Latitude_1'), crs = "epsg:4326")

    facility_table_1 <- calculate_area_taiwan_v1(coord_finder(), 500)
    facility_table_1$Index <- round(sum(facility_table_1[,1:5] * p),2)
    facility_table_1$Price <- round(calculate_estate_price(coord_finder(), 500),2)
    estate_tp_limited_1 <- estate_finder()[which(between(estate_finder()$Price_avg, facility_table_1$Price - 5, facility_table_1$Price + 5)),]
    most_convi <- estate_tp_limited_1[which(estate_tp_limited_1$Index == max(estate_tp_limited_1$Index))[1],c('Latitude', 'Longitude')]
    estate_tp_limited_2 <- estate_finder()[which(between(estate_finder()$Index, facility_table_1$Index - 1.5, facility_table_1$Index + 1.5)),]
    cheapest <- estate_tp_limited_2[which(estate_tp_limited_2$Price_avg == min(estate_tp_limited_2$Price_avg))[1],c('Latitude', 'Longitude')]
    convi_sf <- st_as_sf(most_convi, coords = c('Longitude', 'Latitude'), crs = 4326)
    cheapest_sf <- st_as_sf(cheapest, coords = c('Longitude', 'Latitude'), crs = 4326)
    ggplot(my_sf) +
      geom_sf(aes()) +
      # geom_point(data=facility_finder(), aes(x = Longitude, y = Latitude, color = Facility), size = 0.25, alpha = 0.5)+
      geom_point(data=facilitypreference_finder(), aes(x = Longitude, y = Latitude, color = Facility), size = 0.25, alpha = 0.5)+
      scale_color_manual(name = 'Facility', values = c(c(Store = 'darkkhaki', Hospital = 'pink', Post = 'blue', Police = 'purple', School = 'red')))+
      # geom_sf(data = object_sf, aes(fill = 'Object'), size = 8, alpha = 0.7, shape = 21, stroke = 0, show.legend = 'point')+
      geom_sf(data = convi_sf, aes(fill = 'Most Convenient'), size = 8, alpha = 0.7, shape = 21, stroke = 0, show.legend = 'point')+
      geom_sf(data = cheapest_sf, aes(fill = 'Lowest Price'), size = 8, alpha = 0.7, shape = 21, stroke = 0, show.legend = 'point')+
      scale_fill_manual(name = 'Points', values = c('Object' = 'cyan','Most Convenient' = 'blue','Lowest Price' = 'red'))+
      theme_void()
  })

  output$preferenceFinder <- DT::renderDataTable({
    input$coord_finder
    facility_table_1 <- calculate_area_taiwan_v1(coord_finder(), 500)
    facility_table_1$Index <- round(sum(facility_table_1[,1:5] * p),2)
    facility_table_1$Price <- round(calculate_estate_price(coord_finder(), 500),2)
    facility_table_1$Address <- NA
    facility_table_1$Years <- NA
    estate_tp_limited_1 <- estate_finder()[which(between(estate_finder()$Price_avg, facility_table_1$Price - 5, facility_table_1$Price + 5)),]
    most_convi <- estate_tp_limited_1[which(estate_tp_limited_1$Index == max(estate_tp_limited_1$Index))[1],c('Hospital', 'Store', 'School', 'Police', 'Post', 'Index', 'Price_avg','City', 'Years')]
    estate_tp_limited_2 <- estate_finder()[which(between(estate_finder()$Index, facility_table_1$Index - 1.5, facility_table_1$Index + 1.5)),]
    cheapest <- estate_tp_limited_2[which(estate_tp_limited_2$Price_avg == min(estate_tp_limited_2$Price_avg))[1],c('Hospital', 'Store', 'School', 'Police', 'Post', 'Index', 'Price_avg','City', 'Years')]
    colnames(facility_table_1) <- c('醫院藥局','便利商店','國中小','警察局','郵局', '便利指數', '平均房價/坪', '地址', '屋齡')
    colnames(most_convi) <- c('醫院藥局','便利商店','國中小','警察局','郵局', '便利指數', '平均房價/坪', '地址', '屋齡')
    colnames(cheapest) <- c('醫院藥局','便利商店','國中小','警察局','郵局', '便利指數', '平均房價/坪', '地址', '屋齡')
    preference_table <- rbind(facility_table_1, most_convi, cheapest)
    rownames(preference_table) <- c('原目標', '最方便', '最實惠')
    DT::datatable(preference_table[,c('地址', '醫院藥局','便利商店','國中小','警察局','郵局', '便利指數', '屋齡', '平均房價/坪')])
    # DT::datatable(estate_finder()[,c('Hospital', 'Store', 'School', 'Police', 'Post', 'Index', 'Price_avg')])
  })

  town_finder <- reactive({
    req(input$TownFinder)
    filter(taiwan_town_facility, City_ID %in% input$TownFinder)
  })

  output$townstableFinder <- DT::renderDataTable({
    DT::datatable(town_finder()[,c(1:7)],
                  colnames = c('醫院藥局' = '醫療藥局院所', '國中小' = '學校', '鄉鎮市區' = 'Town'))
  })

}

shinyApp(ui = ui, server = server)